 BAGIAN 1: IMPORT LIBRARY DAN KONFIGURASI AWAL


In [4]:
import tensorflow as tf
import math
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.backends.backend_pdf



In [7]:
# Set style plot Matplotlib
plt.style.use('fivethirtyeight')

# Memeriksa versi TensorFlow
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


 BAGIAN 2: LOAD DATA DAN PRA-PROSES

In [9]:
# Load data dari file CSV
# Asumsi file 'bitcoin.csv' berada di direktori yang sama
try:
    df = pd.read_csv('bitcoin.csv')
    print("Data berhasil dimuat.")
except FileNotFoundError:
    print("File 'bitcoin.csv' tidak ditemukan. Pastikan file ada di direktori yang sama.")
    # Stop eksekusi jika file tidak ditemukan
    raise SystemExit

Data berhasil dimuat.


In [10]:
# Mengubah kolom 'Date' menjadi datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [11]:
# Menampilkan informasi dasar dataset
print("\nInformasi Dataset:")
df.info()
print("\nLima baris data pertama:")
print(df.head())
print(f"\nJumlah baris dan kolom: {df.shape}")


Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1010 entries, 2021-01-01 to 2023-10-07
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1010 non-null   float64
 1   High       1010 non-null   float64
 2   Low        1010 non-null   float64
 3   Close      1010 non-null   float64
 4   Adj Close  1010 non-null   float64
 5   Volume     1010 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 55.2 KB

Lima baris data pertama:
                    Open          High           Low         Close  \
Date                                                                 
2021-01-01  28994.009766  29600.626953  28803.585938  29374.152344   
2021-01-02  29376.455078  33155.117188  29091.181641  32127.267578   
2021-01-03  32129.408203  34608.558594  32052.316406  32782.023438   
2021-01-04  32810.949219  33440.218750  28722.755859  31971.914063   
2021-01-05  31977.041016  34437.589844 

In [12]:
# Menggunakan hanya kolom 'Close' untuk prediksi
close_data = df[['Close']]
print("\nData 'Close' yang akan digunakan:")
print(close_data.head())


Data 'Close' yang akan digunakan:
                   Close
Date                    
2021-01-01  29374.152344
2021-01-02  32127.267578
2021-01-03  32782.023438
2021-01-04  31971.914063
2021-01-05  33992.429688


In [13]:
# Scaling data menggunakan MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_data)
print("\nData 'Close' setelah diskalakan:")
print(scaled_data[:5])


Data 'Close' setelah diskalakan:
[[0.26239837]
 [0.31556831]
 [0.32821338]
 [0.31256803]
 [0.35158953]]


In [14]:
# Membuat dataset training dan testing
training_data_len = math.ceil(len(scaled_data) * .8) # 80% data untuk training
train_data = scaled_data[0:training_data_len, :]
test_data = scaled_data[training_data_len - 60:, :] # Menggunakan 60 hari sebelumnya untuk prediks

In [15]:
# Fungsi untuk membuat dataset dari data deret waktu
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

In [16]:
# Membuat dataset training
X_train, y_train = create_dataset(train_data)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [17]:
# Membuat dataset testing
X_test, y_test = create_dataset(test_data)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [18]:
print(f"\nUkuran data X_train: {X_train.shape}")
print(f"Ukuran data X_test: {X_test.shape}")


Ukuran data X_train: (748, 60, 1)
Ukuran data X_test: (202, 60, 1)


 BAGIAN 3: PEMBANGUNAN DAN PELATIHAN MODEL LSTM

In [19]:
# Membangun model LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Menyimpan model
model.save('model_bitcoin_prediksi_v2.h5')
print("\nModel berhasil dilatih dan disimpan sebagai 'model_bitcoin_prediksi_v2.h5'")

Epoch 1/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - loss: 0.0205 - val_loss: 0.0014
Epoch 2/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0053 - val_loss: 5.8923e-04
Epoch 3/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - loss: 0.0041 - val_loss: 0.0030
Epoch 4/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - loss: 0.0032 - val_loss: 9.0990e-04
Epoch 5/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - loss: 0.0030 - val_loss: 9.7661e-04
Epoch 6/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0036 - val_loss: 5.1388e-04
Epoch 7/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - loss: 0.0029 - val_loss: 3.3119e-04
Epoch 8/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0027 - val_loss: 0.0029
Epoch 9/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 22s 26ms/step - loss: 0.0034 - val_loss: 3.6262e-04
Epoch 10/10
673/673 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 0.0025 - val_loss: 3.9950e-04


In [22]:
# Menyimpan model
model.save('model_bitcoin_prediksi_v2.h5')
print("\nModel berhasil dilatih dan disimpan sebagai 'model_bitcoin_prediksi_v2.h5'")


Model berhasil dilatih dan disimpan sebagai 'model_bitcoin_prediksi_v2.h5'


 BAGIAN 4: EVALUASI DAN VISUALISASI HASIL

In [23]:
# Membuat prediksi
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions) # Kembalikan ke skala harga asli


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step


In [24]:
# Mengembalikan y_test ke skala harga asli
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

In [25]:
# Menghitung metrik evaluasi
rmse = np.sqrt(mean_squared_error(y_test_original, predictions))
mae = mean_absolute_error(y_test_original, predictions)
r2 = r2_score(y_test_original, predictions)
print("\nMetrik Evaluasi:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")


Metrik Evaluasi:
Root Mean Squared Error (RMSE): 871.23
Mean Absolute Error (MAE): 736.53
R-squared (R2): 0.70


In [31]:
# Memvisualisasikan hasil
train = close_data[:training_data_len]
# PENTING: Gunakan .copy() untuk menghindari SettingWithCopyWarning
valid = close_data[training_data_len:].copy()
valid['Predictions'] = predictions

In [32]:
# Plot dengan Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['Close'], name='Data Training', mode='lines'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Close'], name='Data Validasi (Aktual)', mode='lines'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Predictions'], name='Prediksi', mode='lines'))

fig.update_layout(
    title='Model Prediksi LSTM untuk Harga Bitcoin',
    xaxis_title='Tanggal',
    yaxis_title='Harga Penutupan (USD)',
    template='plotly_dark'
)
fig.show()

In [30]:
# Memvisualisasikan history training
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], name='Training Loss'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], name='Validation Loss'))
fig_loss.update_layout(title='Model Loss Selama Pelatihan', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig_loss.show()

BAGIAN 5: PREDIKSI HARGA DI MASA DEPAN

In [33]:
print("\nPrediksi harga untuk 30 hari ke depan...")

# Dapatkan 60 data terakhir dari data yang diskalakan
last_60_days = scaled_data[len(scaled_data) - 60:].copy()

future_predictions = []
current_input = last_60_days
n_future_days = 30

for i in range(n_future_days):
    # Reshape input untuk model LSTM
    x_input = current_input.reshape((1, 60, 1))

    # Lakukan prediksi
    yhat = model.predict(x_input, verbose=0)

    # Tambahkan hasil prediksi ke list
    future_predictions.append(yhat[0, 0])

    # Update input dengan hasil prediksi baru
    current_input = np.append(current_input[1:], yhat[0])

# Kembalikan prediksi ke skala harga asli
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Buat tanggal untuk prediksi masa depan
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=n_future_days)

# Visualisasikan prediksi masa depan
fig_future = go.Figure()

# Plot data historis
fig_future.add_trace(go.Scatter(x=close_data.index, y=close_data['Close'], name='Data Historis', mode='lines'))

# Plot prediksi masa depan
fig_future.add_trace(go.Scatter(x=future_dates, y=future_predictions.flatten(), name='Prediksi Masa Depan', mode='lines', line=dict(color='orange')))

fig_future.update_layout(
    title='Prediksi Harga Bitcoin 30 Hari ke Depan',
    xaxis_title='Tanggal',
    yaxis_title='Harga Penutupan (USD)',
    template='plotly_dark'
)
fig_future.show()


Prediksi harga untuk 30 hari ke depan...
